In [13]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline


In [14]:
def load_preprocessed_images(directory):
    images = []
    for i in range(1, 49):
        path = os.path.join(directory, f"img{i}.jpg")
        img = cv2.imread(path)
        if img is not None:
            images.append(img)
        else:
            print(f"Impossible de charger l'image {path}")
    return images

In [15]:
input_dir = "../data/preprocessed/"
images = load_preprocessed_images(input_dir)

In [16]:
print(f"Nombre d'images chargées : {len(images)}")

Nombre d'images chargées : 48


In [17]:
def stitch_images(images):
    stitcher = cv2.Stitcher.create()
    status, panorama = stitcher.stitch(images)
    
    if status == cv2.Stitcher_OK:
        return panorama
    else:
        print(f"Le stitching a échoué. Code d'erreur : {status}")
        return None

In [18]:
panorama = stitch_images(images)

Le stitching a échoué. Code d'erreur : 3


In [19]:
if panorama is not None:
    plt.figure(figsize=(20, 10))
    plt.imshow(cv2.cvtColor(panorama, cv2.COLOR_BGR2RGB))
    plt.title("Panorama stitché")
    plt.axis('off')
    plt.show()
    
    output_dir = "../output/"
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, "stitched_panorama.jpg")
    cv2.imwrite(output_path, panorama)
    print(f"Panorama sauvegardé dans {output_path}")
else:
    print("Impossible de créer le panorama")

Impossible de créer le panorama


In [20]:
def simple_3d_reconstruction(images):
    sift = cv2.SIFT_create()
    
    keypoints_list = []
    descriptors_list = []
    for img in images:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        kp, des = sift.detectAndCompute(gray, None)
        keypoints_list.append(kp)
        descriptors_list.append(des)
    
    matcher = cv2.BFMatcher()
    transformations = []
    for i in range(len(images) - 1):
        matches = matcher.knnMatch(descriptors_list[i], descriptors_list[i+1], k=2)
        good_matches = [m for m, n in matches if m.distance < 0.7 * n.distance]
        
        if len(good_matches) > 10:
            src_pts = np.float32([keypoints_list[i][m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
            dst_pts = np.float32([keypoints_list[i+1][m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
            M, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
            transformations.append(M)
        else:
            print(f"Pas assez de bons matches entre les images {i} et {i+1}")
            transformations.append(np.eye(3))
    
    return transformations


In [21]:
transformations = simple_3d_reconstruction(images)

In [22]:
output_dir = "../output/"
os.makedirs(output_dir, exist_ok=True)

In [23]:
output_path = os.path.join(output_dir, "transformations.npy")
np.save(output_path, transformations)
print(f"Transformations sauvegardées dans {output_path}")

Transformations sauvegardées dans ../output/transformations.npy
